In [ ]:
%%sh
wget -N https://sagemaker-sample-data-us-west-2.s3-us-west-2.amazonaws.com/autopilot/direct_marketing/bank-additional.zip
unzip -o bank-additional.zip

In [ ]:
import sagemaker

print(sagemaker.__version__)

sess   = sagemaker.Session()
bucket = sess.default_bucket()                     
prefix = 'xgboost-direct-marketing'

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

data = pd.read_csv('./bank-additional/bank-additional-full.csv')

# One-hot encode
data = pd.get_dummies(data)

# Move labels to first column, which is what XGBoost expects
data = data.drop(['y_no'], axis=1)
data = pd.concat([data['y_yes'], data.drop(['y_yes'], axis=1)], axis=1)

# Shuffle and split into training and validation (95%/5%)
data = data.sample(frac=1, random_state=123)
train_data, val_data = train_test_split(data, test_size=0.05)

# Save to CSV files
train_data.to_csv('training.csv', index=False, header=True) # Need to keep column names
val_data.to_csv('validation.csv', index=False, header=True)

In [ ]:
# Use this cell for local mode
training = 'file://training.csv'
validation = 'file://validation.csv'
output   = 'file:///tmp'

train_instance_type = deploy_instance_type = 'local'

In [ ]:
# Use this cell for managed mode
training = sess.upload_data(path='training.csv', key_prefix=prefix + "/training")
validation = sess.upload_data(path="validation.csv", key_prefix=prefix + "/validation")
output   = 's3://{}/{}/output/'.format(bucket,prefix)

train_instance_type = 'ml.m5.large'
deploy_instance_type = 'ml.t2.medium'

In [ ]:
print(training)
print(validation)
print(output)

# Train and deploy on SageMaker

In [ ]:
from sagemaker.xgboost import XGBoost

role = sagemaker.get_execution_role()
#role = 'arn:aws:iam::0123456789012:role/Sagemaker-fullaccess'

xgb_estimator = XGBoost(entry_point='xgb-dm.py', 
                          role=role,
                          instance_count=1, 
                          instance_type=train_instance_type,
                          framework_version='1.0-1',
                          py_version='py3',
                          output_path=output,
                          hyperparameters={
                              'max-depth': 5,
                              'eval-metric': 'auc'
                          })

In [ ]:
xgb_estimator.fit({'training':training, 'validation':validation})

In [ ]:
import time

xgb_endpoint_name = prefix+time.strftime("%Y-%m-%d-%H-%M-%S", time.gmtime())

xgb_predictor = xgb_estimator.deploy(
                     endpoint_name=xgb_endpoint_name,
                     initial_instance_count=1, 
                     instance_type=deploy_instance_type)

print(xgb_endpoint_name)

In [ ]:
# Load some samples, drop labels, and one-hot encode
payload = val_data[:10].drop(['y_yes'], axis=1)
payload = payload.to_csv(header=False, index=False).rstrip('\n')
print(payload)

In [ ]:
xgb_predictor.content_type = 'text/csv'
xgb_predictor.serializer = sagemaker.serializers.CSVSerializer()
xgb_predictor.deserializer = sagemaker.deserializers.CSVDeserializer()

response = xgb_predictor.predict(payload)

print(response)

In [ ]:
xgb_predictor.delete_endpoint()